In [1]:
import librosa
import numpy as np
import os
import math
from sklearn.cluster import KMeans
import hmmlearn.hmm
from operator import itemgetter, attrgetter

def get_mfcc(file_path):
    y, sr = librosa.load(file_path) # read .wav file
    hop_length = math.floor(sr*0.010) # 10ms hop
    win_length = math.floor(sr*0.025) # 25ms frame
    # mfcc is 12 x T matrix
    mfcc = librosa.feature.mfcc(
        y, sr, n_mfcc=12, n_fft=1024,
        hop_length=hop_length, win_length=win_length)
    # substract mean from mfcc --> normalize mfcc
    mfcc = mfcc - np.mean(mfcc, axis=1).reshape((-1,1)) 
    # delta feature 1st order and 2nd order
    delta1 = librosa.feature.delta(mfcc, order=1)
    delta2 = librosa.feature.delta(mfcc, order=2)
    # X is 36 x T
    X = np.concatenate([mfcc, delta1, delta2], axis=0) # O^r
    # return T x 36 (transpose of X)
    return X.T # hmmlearn use T x N matrix

def get_class_data(data_dir):
    files = os.listdir(data_dir)
    mfcc = [get_mfcc(os.path.join(data_dir,f)) for f in files if f.endswith(".wav")]
    return mfcc

def clustering(X, n_clusters=10):
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0, verbose=0)
    kmeans.fit(X)
    print("centers", kmeans.cluster_centers_.shape)
    return kmeans

In [2]:
if __name__ == "__main__":
    train_names = ["Nguoi","Duoc","Cothe","Trong","Dang"]
    test_names = ["test_Cothe","test_Nguoi","test_Duoc","test_Dang","test_Trong"]
    train_dataset = {}
    test_dataset ={}
    for cname in train_names:
        print(f"Load {cname} dataset")
        train_dataset[cname] = get_class_data(os.path.join("Data", cname))
        
    for cname in test_names:
        print(f"Load {cname} dataset")
        test_dataset[cname] = get_class_data(os.path.join("Data", cname))

    # Get all vectors in the train_dataset
    all_vectors = np.concatenate([np.concatenate(v, axis=0) for k, v in train_dataset.items()], axis=0)
    print("vectors", all_vectors.shape)
    # Run K-Means algorithm to get clusters
    kmeans = clustering(all_vectors)
    print("centers", kmeans.cluster_centers_.shape)
    
    for cname in train_dataset:
        train_dataset[cname] = list([kmeans.predict(v).reshape(-1, 1) for v in train_dataset[cname]])
    for cname in test_dataset:
        test_dataset[cname] = list([kmeans.predict(v).reshape(-1, 1) for v in test_dataset[cname]])
    
    models = {}

Load Nguoi dataset
Load Duoc dataset
Load Cothe dataset
Load Trong dataset
Load Dang dataset
Load test_Cothe dataset
Load test_Nguoi dataset
Load test_Duoc dataset
Load test_Dang dataset
Load test_Trong dataset
vectors (11440, 36)
centers (10, 36)
centers (10, 36)


In [3]:

    cname = 'Nguoi'
    hmm = hmmlearn.hmm.MultinomialHMM(n_components=12, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
    hmm.transmat_ =np.array([
        [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])
    if cname[:4] != 'test':
        X = np.concatenate(train_dataset[cname])
        lengths = list([len(x) for x in train_dataset[cname]])
        print("training class", cname)
        print(X.shape, lengths, len(lengths))
        hmm.fit(X, lengths=lengths)
        models[cname] = hmm
    print("Training done")

    

training class Nguoi
(3536, 1) [21, 18, 22, 15, 21, 30, 20, 28, 19, 16, 12, 14, 27, 23, 21, 17, 33, 14, 27, 23, 22, 15, 12, 16, 16, 41, 20, 46, 16, 39, 20, 26, 16, 15, 20, 19, 27, 17, 16, 22, 28, 20, 23, 25, 20, 19, 16, 39, 17, 26, 28, 28, 31, 22, 38, 18, 40, 18, 28, 28, 16, 20, 15, 19, 1282, 12, 17, 18, 46, 17, 21, 14, 19, 26, 39, 18, 16, 24, 22, 27, 15, 22, 17, 26, 36, 32, 25, 19, 30, 21, 25, 20, 21, 23, 19, 24, 36, 26, 21, 16] 100


         1       -7894.9260             +nan
         2       -5672.7265       +2222.1995
         3       -5189.6997        +483.0268
         4       -4963.6770        +226.0227
         5       -4857.7628        +105.9142
         6       -4830.2674         +27.4954
         7       -4819.5501         +10.7173
         8       -4811.0260          +8.5241
         9       -4803.5395          +7.4866
        10       -4797.2804          +6.2591
        11       -4791.3022          +5.9782
        12       -4784.3083          +6.9939
        13       -4779.3627          +4.9456
        14       -4776.8608          +2.5019
        15       -4775.2841          +1.5767
        16       -4774.1446          +1.1394
        17       -4773.3087          +0.8360
        18       -4772.6453          +0.6634
        19       -4771.9896          +0.6558
        20       -4771.1839          +0.8056
        21       -4770.0793          +1.1046
        22       -4768.5789          +1.5004
        23

Training done


        57       -4753.9823          +0.0377
        58       -4753.9655          +0.0168
        59       -4753.9575          +0.0080


In [4]:
    cname = 'Duoc'
    hmm = hmmlearn.hmm.MultinomialHMM(n_components=12, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
    hmm.transmat_ =np.array([
        [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])
    if cname[:4] != 'test':
        X = np.concatenate(train_dataset[cname])
        lengths = list([len(x) for x in train_dataset[cname]])
        print("training class", cname)
        print(X.shape, lengths, len(lengths))
        hmm.fit(X, lengths=lengths)
        models[cname] = hmm
    print("Training done")

         1       -2704.6712             +nan


training class Duoc
(1192, 1) [20, 15, 13, 17, 36, 16, 16, 20, 23, 18, 16, 20, 26, 24, 17, 17, 34, 21, 21, 21, 16, 15, 13, 16, 17, 16, 11, 12, 17, 21, 20, 11, 17, 16, 11, 13, 15, 16, 14, 14, 17, 17, 13, 10, 15, 23, 14, 11, 18, 17, 24, 14, 18, 9, 11, 14, 16, 15, 20, 14, 15, 19, 17, 14, 16, 21, 15, 14, 24, 15] 70


         2       -2080.9031        +623.7681
         3       -1905.1311        +175.7720
         4       -1817.0019         +88.1292
         5       -1775.0706         +41.9313
         6       -1753.7746         +21.2960
         7       -1742.5459         +11.2287
         8       -1735.7120          +6.8339
         9       -1730.7986          +4.9135
        10       -1727.0508          +3.7478
        11       -1724.2838          +2.7670
        12       -1722.2115          +2.0723
        13       -1720.3243          +1.8872
        14       -1717.5894          +2.7349
        15       -1713.3451          +4.2443
        16       -1708.5846          +4.7605
        17       -1705.1989          +3.3857
        18       -1703.6249          +1.5740
        19       -1702.7889          +0.8360
        20       -1702.2246          +0.5643
        21       -1701.7634          +0.4612
        22       -1701.3102          +0.4533
        23       -1700.6802          +0.6299
        24

Training done


        98       -1687.8209          +0.0123
        99       -1687.8108          +0.0101
       100       -1687.8025          +0.0084


In [5]:
    cname = 'Cothe'
    hmm = hmmlearn.hmm.MultinomialHMM(n_components=12, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
    hmm.transmat_ =np.array([
        [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])
    if cname[:4] != 'test':
        X = np.concatenate(train_dataset[cname])
        lengths = list([len(x) for x in train_dataset[cname]])
        print("training class", cname)
        print(X.shape, lengths, len(lengths))
        hmm.fit(X, lengths=lengths)
        models[cname] = hmm
    print("Training done")

         1       -8265.1031             +nan


training class Cothe
(3860, 1) [51, 31, 23, 36, 24, 35, 44, 33, 48, 32, 51, 32, 52, 42, 39, 27, 54, 46, 37, 36, 30, 46, 41, 34, 47, 37, 29, 40, 31, 29, 33, 37, 46, 48, 36, 34, 32, 30, 44, 61, 36, 39, 35, 40, 74, 37, 60, 70, 35, 49, 40, 35, 54, 45, 40, 32, 31, 48, 83, 36, 56, 31, 40, 43, 46, 44, 38, 30, 32, 29, 39, 29, 36, 35, 34, 34, 35, 31, 39, 32, 32, 41, 45, 46, 29, 35, 33, 40, 47, 36, 31, 28, 26, 33, 26, 31, 29, 25, 23, 34] 100


         2       -6051.1840       +2213.9191
         3       -5505.0515        +546.1325
         4       -5325.0825        +179.9690
         5       -5229.4499         +95.6326
         6       -5164.8648         +64.5851
         7       -5110.2423         +54.6225
         8       -5076.0494         +34.1929
         9       -5056.0200         +20.0294
        10       -5043.0311         +12.9890
        11       -5030.8691         +12.1619
        12       -5014.7590         +16.1101
        13       -4994.6104         +20.1486
        14       -4974.6915         +19.9188
        15       -4955.8682         +18.8234
        16       -4941.8762         +13.9920
        17       -4937.4182          +4.4580
        18       -4935.5772          +1.8410
        19       -4933.4966          +2.0806
        20       -4931.7524          +1.7442
        21       -4930.9496          +0.8028
        22       -4930.5784          +0.3712
        23       -4930.3787          +0.1996
        24

Training done


        49       -4926.3258          +0.0098


In [6]:
    cname = 'Trong'
    hmm = hmmlearn.hmm.MultinomialHMM(n_components=9, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
    hmm.transmat_ =np.array([
        [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])
    if cname[:4] != 'test':
        X = np.concatenate(train_dataset[cname])
        lengths = list([len(x) for x in train_dataset[cname]])
        print("training class", cname)
        print(X.shape, lengths, len(lengths))
        hmm.fit(X, lengths=lengths)
        models[cname] = hmm
    print("Training done")

         1       -3445.9545             +nan
         2       -2100.8539       +1345.1006
         3       -1826.7356        +274.1183
         4       -1765.1053         +61.6304
         5       -1731.1688         +33.9365
         6       -1697.5269         +33.6419


training class Trong
(1506, 1) [20, 20, 21, 28, 23, 25, 24, 25, 23, 16, 15, 16, 24, 20, 19, 22, 26, 23, 14, 12, 18, 23, 14, 18, 34, 24, 17, 14, 19, 15, 18, 29, 22, 22, 21, 23, 18, 24, 34, 33, 30, 23, 19, 12, 19, 23, 25, 22, 19, 16, 26, 21, 20, 18, 24, 28, 20, 20, 28, 21, 21, 21, 25, 25, 25, 21, 21, 19, 22, 16] 70


         7       -1661.0326         +36.4943
         8       -1630.5950         +30.4376
         9       -1613.0326         +17.5624
        10       -1605.3277          +7.7049
        11       -1602.1849          +3.1428
        12       -1600.7477          +1.4372
        13       -1599.6657          +1.0820
        14       -1597.1269          +2.5388
        15       -1595.5281          +1.5988
        16       -1594.9950          +0.5331
        17       -1594.7463          +0.2487
        18       -1594.6133          +0.1330
        19       -1594.5335          +0.0798
        20       -1594.4803          +0.0532
        21       -1594.4417          +0.0386
        22       -1594.4116          +0.0301
        23       -1594.3865          +0.0251
        24       -1594.3641          +0.0224
        25       -1594.3424          +0.0216
        26       -1594.3199          +0.0225
        27       -1594.2949          +0.0251
        28       -1594.2653          +0.0295
        29

Training done


        45       -1593.1313          +0.0158
        46       -1593.1191          +0.0122
        47       -1593.1093          +0.0098


In [7]:
    cname = 'Dang'
    hmm = hmmlearn.hmm.MultinomialHMM(n_components=12, random_state=0, n_iter=1000, verbose=True, init_params='e', params='ste')
    hmm.startprob_ = np.array([0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ])
    hmm.transmat_ =np.array([
        [0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, 0.0, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.2, 0.1, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7, 0.3, ],
        [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ],])
    if cname[:4] != 'test':
        X = np.concatenate(train_dataset[cname])
        lengths = list([len(x) for x in train_dataset[cname]])
        print("training class", cname)
        print(X.shape, lengths, len(lengths))
        hmm.fit(X, lengths=lengths)
        models[cname] = hmm
    print("Training done")

         1       -2988.5260             +nan
         2       -2415.3620        +573.1640
         3       -2165.5582        +249.8037


training class Dang
(1346, 1) [22, 23, 13, 16, 21, 15, 24, 15, 22, 16, 26, 17, 11, 25, 24, 25, 18, 14, 18, 17, 18, 18, 14, 18, 25, 25, 21, 14, 13, 20, 20, 33, 17, 22, 14, 26, 29, 20, 21, 13, 18, 22, 22, 18, 17, 24, 18, 27, 15, 21, 18, 16, 16, 18, 18, 23, 20, 19, 19, 12, 20, 24, 17, 14, 12, 19, 19, 13, 22, 22] 70


         4       -2043.9836        +121.5747
         5       -1960.0322         +83.9514
         6       -1897.2084         +62.8238
         7       -1851.0429         +46.1655
         8       -1836.7105         +14.3324
         9       -1831.0145          +5.6961
        10       -1826.0285          +4.9859
        11       -1819.8666          +6.1620
        12       -1814.8521          +5.0144
        13       -1810.8027          +4.0494
        14       -1805.6662          +5.1365
        15       -1801.8584          +3.8078
        16       -1799.4978          +2.3606
        17       -1797.6577          +1.8402
        18       -1796.2055          +1.4521
        19       -1795.0369          +1.1686
        20       -1794.0704          +0.9665
        21       -1793.2430          +0.8274
        22       -1792.5065          +0.7366
        23       -1791.8269          +0.6795
        24       -1791.1850          +0.6419
        25       -1790.5742          +0.6108
        26

Training done


        81       -1771.2611          +0.0149
        82       -1771.2484          +0.0127
        83       -1771.2375          +0.0109
        84       -1771.2282          +0.0093


In [8]:
    print("Testing")
    test_number = {}
    test_correct = {}
    for true_cname in test_names:
        total = 0
        sum = 0
        print(true_cname)
        for O in test_dataset[true_cname]:
            total += 1
            scores = {}
            for cname, model in models.items():
                score = model.score(O, [len(O)])
                scores[cname] = score
            srt = sorted(scores.items(), key=lambda x: x[1], reverse=True)
            print(srt)
            print(true_cname[5:])
            key , value = srt[0]
            if key == true_cname[5:]:
                sum += 1
        test_number[true_cname[5:]] = total
        test_correct[true_cname[5:]] = sum
        print(test_number[true_cname[5:]])
    
    #test_ = ["Cothe","Nguoi"]   
    for cname in train_names:
        print('Accuracy:', cname, test_correct[cname]/test_number[cname])
    
    print(test_number)

Testing
test_Cothe
[('Cothe', -31.490586554411426), ('Nguoi', -50.94884877223208), ('Duoc', -62.79661033822374), ('Trong', -171.13754459712646), ('Dang', -432.7686835476112)]
Cothe
[('Cothe', -29.214581349991597), ('Duoc', -31.04046758084527), ('Nguoi', -32.53112086450465), ('Dang', -95.2209572272903), ('Trong', -205.6348844834126)]
Cothe
[('Nguoi', -21.132808976141774), ('Cothe', -21.397768931842926), ('Duoc', -31.663061850386903), ('Dang', -57.943281411710394), ('Trong', -190.35620700155306)]
Cothe
[('Cothe', -45.04913840518471), ('Nguoi', -63.24115924706044), ('Duoc', -72.19391745255618), ('Dang', -245.4300599972984), ('Trong', -832.8871833791818)]
Cothe
[('Cothe', -44.20820318967956), ('Nguoi', -45.2936276846084), ('Duoc', -51.36772459144371), ('Trong', -88.03923891280141), ('Dang', -158.91349383324413)]
Cothe
[('Nguoi', -29.606021259159867), ('Cothe', -31.96554082133876), ('Duoc', -46.530037061214486), ('Dang', -175.27746505364055), ('Trong', -251.55844669155692)]
Cothe
[('Cothe',

In [9]:
np.around(models['Nguoi'].transmat_, 2)

array([[0.74, 0.25, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.75, 0.08, 0.17, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.86, 0.03, 0.1 , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.77, 0.18, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.72, 0.27, 0.01, 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.65, 0.09, 0.26, 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.66, 0.  , 0.34, 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.62, 0.37, 0.01, 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.78, 0.22, 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.01,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99,
        0.01],
       [0.  , 0.  , 0

In [10]:
np.around(models['Cothe'].transmat_, 2)

array([[0.81, 0.17, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.86, 0.1 , 0.04, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.74, 0.19, 0.07, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.7 , 0.25, 0.05, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.82, 0.04, 0.14, 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.82, 0.18, 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.83, 0.15, 0.02, 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.79, 0.21, 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.25, 0.05,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.86, 0.05,
        0.09],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ,
        0.  ],
       [0.  , 0.  , 0

In [11]:
np.around(models['Trong'].transmat_, 2)

array([[0.82, 0.16, 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.63, 0.37, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.73, 0.26, 0.01, 0.  , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.7 , 0.02, 0.28, 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.6 , 0.4 , 0.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.7 , 0.3 , 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.67, 0.13, 0.2 ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.79, 0.21],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]])

In [12]:
np.around(models['Duoc'].transmat_, 2)

array([[0.65, 0.26, 0.09, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.83, 0.17, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.55, 0.45, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.16, 0.41, 0.43, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.11, 0.82, 0.07, 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.55, 0.05, 0.4 , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.4 , 0.6 , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.46, 0.45, 0.09,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.59, 0.41,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.51,
        0.49],
       [0.  , 0.  , 0

In [13]:
np.around(models['Dang'].transmat_, 2)

array([[0.74, 0.26, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.78, 0.2 , 0.02, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.75, 0.24, 0.01, 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.63, 0.02, 0.35, 0.  , 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.77, 0.01, 0.22, 0.  , 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.77, 0.08, 0.16, 0.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.84, 0.12, 0.04, 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.8 , 0.03, 0.17,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  ,
        0.  ],
       [0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
        1.  ],
       [0.  , 0.  , 0